# Lecture 4: Quantification of Single-Cell RNA-seq Data - SOLUTION

**Course:** Single-Cell Neurogenomics  
**Date:** December 13, 2025

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc

sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(8, 6))
print("Libraries loaded successfully!")

---

## Task 1: Understanding Count Matrix Structure (20 points)

In [ ]:
# Load PBMC 3k dataset
adata = sc.datasets.pbmc3k()

print("="*70)
print("Count Matrix Structure")
print("="*70)
print(f"Number of cells (observations): {adata.n_obs}")
print(f"Number of genes (variables): {adata.n_vars}")
print(f"Matrix shape: {adata.X.shape}")
print(f"Matrix type: {type(adata.X)}")
print()

# Calculate sparsity
if hasattr(adata.X, 'nnz'):
    sparsity = 1 - (adata.X.nnz / (adata.n_obs * adata.n_vars))
    print(f"Sparsity: {sparsity*100:.2f}%")
print()

# Calculate total counts per cell
total_counts = np.array(adata.X.sum(axis=1)).flatten()
print(f"Total UMI counts per cell:")
print(f"  Mean: {total_counts.mean():.0f}")
print(f"  Median: {np.median(total_counts):.0f}")
print(f"  Min: {total_counts.min():.0f}")
print(f"  Max: {total_counts.max():.0f}")
print()

# Distribution of total counts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(total_counts, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(total_counts.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {total_counts.mean():.0f}')
axes[0].set_xlabel('Total UMI Counts', fontsize=12)
axes[0].set_ylabel('Number of Cells', fontsize=12)
axes[0].set_title('Distribution of Total Counts per Cell', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Barcode rank plot
sorted_counts = np.sort(total_counts)[::-1]
ranks = np.arange(1, len(sorted_counts) + 1)

axes[1].plot(ranks, sorted_counts, linewidth=2, color='darkblue')
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel('Cell Rank (log scale)', fontsize=12)
axes[1].set_ylabel('Total UMI Counts (log scale)', fontsize=12)
axes[1].set_title('Barcode Rank Plot', fontsize=13, fontweight='bold')
axes[1].grid(alpha=0.3, which='both')

# Add knee point annotation
knee_idx = int(0.1 * len(sorted_counts))  # Approximate knee
axes[1].axvline(ranks[knee_idx], color='red', linestyle='--', alpha=0.7, label='Approximate knee')
axes[1].axhline(sorted_counts[knee_idx], color='red', linestyle='--', alpha=0.7)
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nBarcode Rank Plot Interpretation:")
print("- The 'knee' separates high-quality cells from empty droplets")
print("- Steep drop indicates clear distinction between cells and background")
print(f"- Estimated ~{knee_idx} high-quality cells")

---

## Task 2: Interpreting QC Metrics (25 points)

In [ ]:
# Identify mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('MT-')
print(f"Mitochondrial genes found: {adata.var['mt'].sum()}")
print(f"MT genes: {', '.join(adata.var_names[adata.var['mt']])}")
print()

# Calculate QC metrics
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

print("QC Metrics Summary:")
print("="*70)
print(adata.obs[['n_genes_by_counts', 'total_counts', 'pct_counts_mt']].describe())
print()

# Create comprehensive QC visualizations
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Total counts violin
sc.pl.violin(adata, 'total_counts', ax=axes[0, 0], show=False)
axes[0, 0].set_title('Total UMI Counts per Cell', fontweight='bold')

# 2. Genes detected violin
sc.pl.violin(adata, 'n_genes_by_counts', ax=axes[0, 1], show=False)
axes[0, 1].set_title('Genes Detected per Cell', fontweight='bold')

# 3. Mitochondrial % violin
sc.pl.violin(adata, 'pct_counts_mt', ax=axes[0, 2], show=False)
axes[0, 2].set_title('Mitochondrial Gene %', fontweight='bold')

# 4. Total counts vs genes detected
axes[1, 0].scatter(adata.obs['total_counts'], adata.obs['n_genes_by_counts'], 
                   alpha=0.5, s=10, c='steelblue')
axes[1, 0].set_xlabel('Total UMI Counts')
axes[1, 0].set_ylabel('Genes Detected')
axes[1, 0].set_title('Counts vs Genes Detected', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 5. Total counts vs MT%
axes[1, 1].scatter(adata.obs['total_counts'], adata.obs['pct_counts_mt'], 
                   alpha=0.5, s=10, c='coral')
axes[1, 1].set_xlabel('Total UMI Counts')
axes[1, 1].set_ylabel('Mitochondrial %')
axes[1, 1].set_title('Counts vs Mitochondrial %', fontweight='bold')
axes[1, 1].axhline(y=5, color='red', linestyle='--', label='5% threshold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

# 6. Genes vs MT%
axes[1, 2].scatter(adata.obs['n_genes_by_counts'], adata.obs['pct_counts_mt'], 
                   alpha=0.5, s=10, c='mediumseagreen')
axes[1, 2].set_xlabel('Genes Detected')
axes[1, 2].set_ylabel('Mitochondrial %')
axes[1, 2].set_title('Genes vs Mitochondrial %', fontweight='bold')
axes[1, 2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Identify potential quality thresholds
print("\nRecommended Quality Thresholds:")
print("="*70)
print(f"Minimum genes: 200 (remove low-quality cells)")
print(f"Minimum counts: 500 (remove debris)")
print(f"Maximum counts: {int(adata.obs['total_counts'].quantile(0.98))} (98th percentile, remove doublets)")
print(f"Maximum MT%: 5% (remove dying cells)")
print()

# Calculate impact
n_before = adata.n_obs
filter_mask = (
    (adata.obs['n_genes_by_counts'] >= 200) &
    (adata.obs['total_counts'] >= 500) &
    (adata.obs['total_counts'] <= adata.obs['total_counts'].quantile(0.98)) &
    (adata.obs['pct_counts_mt'] <= 5)
)
n_after = filter_mask.sum()
print(f"Cells before filtering: {n_before}")
print(f"Cells after filtering: {n_after}")
print(f"Cells removed: {n_before - n_after} ({(n_before - n_after)/n_before*100:.1f}%)")

---

## Task 3: Sequencing Saturation Analysis (20 points)

In [ ]:
# Simulate sequencing saturation by subsampling
print("Sequencing Saturation Analysis")
print("="*70)

# Select a subset of cells for analysis (for speed)
np.random.seed(42)
cell_subset = np.random.choice(adata.n_obs, min(500, adata.n_obs), replace=False)
adata_subset = adata[cell_subset].copy()

# Test different subsampling depths
sampling_fractions = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
mean_genes_detected = []

for frac in sampling_fractions:
    # Subsample counts
    X_sub = adata_subset.X.copy()
    if hasattr(X_sub, 'toarray'):
        X_sub = X_sub.toarray()
    
    # Binomial subsampling
    X_sampled = np.random.binomial(X_sub.astype(int), frac)
    
    # Count genes detected
    genes_per_cell = (X_sampled > 0).sum(axis=1)
    mean_genes_detected.append(genes_per_cell.mean())
    print(f"Depth {frac*100:3.0f}%: {genes_per_cell.mean():.0f} genes detected on average")

print()

# Plot saturation curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Saturation curve
axes[0].plot(sampling_fractions, mean_genes_detected, 'o-', linewidth=2, markersize=8, color='steelblue')
axes[0].set_xlabel('Sequencing Depth (fraction)', fontsize=12)
axes[0].set_ylabel('Mean Genes Detected', fontsize=12)
axes[0].set_title('Sequencing Saturation Curve', fontsize=13, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].axhline(y=mean_genes_detected[-1]*0.95, color='red', linestyle='--', 
                label='95% saturation', alpha=0.7)
axes[0].legend()

# Marginal gain
marginal_gain = np.diff(mean_genes_detected)
axes[1].plot(sampling_fractions[1:], marginal_gain, 'o-', linewidth=2, markersize=8, color='coral')
axes[1].set_xlabel('Sequencing Depth (fraction)', fontsize=12)
axes[1].set_ylabel('Marginal Genes Gained', fontsize=12)
axes[1].set_title('Marginal Gain per Additional Depth', fontsize=13, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Interpretation
print("\nInterpretation:")
print("="*70)
saturation_95 = mean_genes_detected[-1] * 0.95
depth_for_95 = None
for i, genes in enumerate(mean_genes_detected):
    if genes >= saturation_95:
        depth_for_95 = sampling_fractions[i]
        break

if depth_for_95:
    print(f"✓ Sequencing is well-saturated")
    print(f"✓ 95% saturation achieved at ~{depth_for_95*100:.0f}% depth")
    print(f"✓ Additional sequencing provides diminishing returns")
else:
    print(f"⚠ Sequencing may benefit from additional depth")
    print(f"⚠ Curve has not plateaued")

---

## Task 4: Gene Detection Analysis (20 points)

In [ ]:
# Calculate how many cells each gene is detected in
print("Gene Detection Analysis")
print("="*70)

# Get detection per gene
if hasattr(adata.X, 'toarray'):
    X_dense = adata.X.toarray()
else:
    X_dense = adata.X

cells_per_gene = (X_dense > 0).sum(axis=0)
pct_cells_per_gene = (cells_per_gene / adata.n_obs) * 100

# Add to var
adata.var['n_cells'] = cells_per_gene
adata.var['pct_cells'] = pct_cells_per_gene

# Categorize genes
highly_detected = adata.var_names[adata.var['pct_cells'] > 50]
rarely_detected = adata.var_names[adata.var['pct_cells'] < 1]

print(f"Total genes: {adata.n_vars}")
print(f"Highly detected genes (>50% cells): {len(highly_detected)}")
print(f"Rarely detected genes (<1% cells): {len(rarely_detected)}")
print()

print("Top 20 most widely detected genes:")
top_genes = adata.var.nlargest(20, 'pct_cells')[['n_cells', 'pct_cells']]
print(top_genes)
print()

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Distribution of detection
axes[0, 0].hist(pct_cells_per_gene, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('% of Cells Expressing Gene', fontsize=11)
axes[0, 0].set_ylabel('Number of Genes', fontsize=11)
axes[0, 0].set_title('Distribution of Gene Detection', fontsize=12, fontweight='bold')
axes[0, 0].axvline(50, color='red', linestyle='--', label='50% threshold')
axes[0, 0].axvline(1, color='orange', linestyle='--', label='1% threshold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Log-scale distribution
axes[0, 1].hist(np.log10(cells_per_gene + 1), bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Log10(Number of Cells + 1)', fontsize=11)
axes[0, 1].set_ylabel('Number of Genes', fontsize=11)
axes[0, 1].set_title('Gene Detection (Log Scale)', fontsize=12, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# 3. Cumulative distribution
sorted_pct = np.sort(pct_cells_per_gene)
cumulative = np.arange(1, len(sorted_pct) + 1) / len(sorted_pct) * 100
axes[1, 0].plot(sorted_pct, cumulative, linewidth=2, color='mediumseagreen')
axes[1, 0].set_xlabel('% of Cells Expressing Gene', fontsize=11)
axes[1, 0].set_ylabel('Cumulative % of Genes', fontsize=11)
axes[1, 0].set_title('Cumulative Gene Detection', fontsize=12, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 4. Top genes bar plot
top_20_genes = adata.var.nlargest(20, 'pct_cells')
axes[1, 1].barh(range(20), top_20_genes['pct_cells'][::-1], color='steelblue', edgecolor='black')
axes[1, 1].set_yticks(range(20))
axes[1, 1].set_yticklabels(top_20_genes.index[::-1], fontsize=9)
axes[1, 1].set_xlabel('% of Cells', fontsize=11)
axes[1, 1].set_title('Top 20 Widely Detected Genes', fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("="*70)
print(f"- Highly detected genes ({len(highly_detected)}) are often housekeeping genes")
print(f"- Rarely detected genes ({len(rarely_detected)}) may be:")
print(f"  • Cell-type-specific markers")
print(f"  • Technical noise/artifacts")
print(f"  • Genes to consider filtering (<3 cells)")

---

## Task 5: Preparing Data for Analysis (15 points)

In [ ]:
# Filter cells and genes
print("Filtering Data for Downstream Analysis")
print("="*70)

# Store original dimensions
n_cells_before = adata.n_obs
n_genes_before = adata.n_vars

print(f"Before filtering:")
print(f"  Cells: {n_cells_before}")
print(f"  Genes: {n_genes_before}")
print()

# Filter cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_cells(adata, min_counts=500)

# Filter by MT% and max counts
adata = adata[adata.obs['pct_counts_mt'] < 5, :].copy()
max_counts_threshold = np.quantile(adata.obs['total_counts'], 0.98)
adata = adata[adata.obs['total_counts'] < max_counts_threshold, :].copy()

print(f"After cell filtering:")
print(f"  Cells: {adata.n_obs}")
print(f"  Cells removed: {n_cells_before - adata.n_obs} ({(n_cells_before - adata.n_obs)/n_cells_before*100:.1f}%)")
print()

# Filter genes
sc.pp.filter_genes(adata, min_cells=3)

print(f"After gene filtering:")
print(f"  Genes: {adata.n_vars}")
print(f"  Genes removed: {n_genes_before - adata.n_vars} ({(n_genes_before - adata.n_vars)/n_genes_before*100:.1f}%)")
print()

# Compare before/after
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Recalculate QC metrics
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# Total counts
axes[0].hist(adata.obs['total_counts'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Total Counts', fontsize=11)
axes[0].set_ylabel('Number of Cells', fontsize=11)
axes[0].set_title('Total Counts (After Filtering)', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)

# Genes detected
axes[1].hist(adata.obs['n_genes_by_counts'], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Genes Detected', fontsize=11)
axes[1].set_ylabel('Number of Cells', fontsize=11)
axes[1].set_title('Genes Detected (After Filtering)', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

# MT%
axes[2].hist(adata.obs['pct_counts_mt'], bins=50, color='mediumseagreen', edgecolor='black', alpha=0.7)
axes[2].set_xlabel('Mitochondrial %', fontsize=11)
axes[2].set_ylabel('Number of Cells', fontsize=11)
axes[2].set_title('Mitochondrial % (After Filtering)', fontsize=12, fontweight='bold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("\nFiltered Dataset Summary:")
print("="*70)
print(adata.obs[['n_genes_by_counts', 'total_counts', 'pct_counts_mt']].describe())
print()

print("Data is now ready for downstream analysis!")
print(f"Final dataset: {adata.n_obs} cells × {adata.n_vars} genes")

---

## Summary

In this assignment, you learned to:
- ✓ Understand count matrix structure and sparsity
- ✓ Create and interpret barcode rank plots
- ✓ Calculate and visualize QC metrics
- ✓ Analyze sequencing saturation
- ✓ Evaluate gene detection patterns
- ✓ Filter data for downstream analysis

These quantification and QC steps are critical foundations for all downstream single-cell analyses!